In [34]:
import pandas as pd
from cleaning import read_data, clean_linting_results, clean_review_test_results, clean_data_reviewss, clean_data_exercises, clean_data_code_blast_tests

read_linting_results, read_review_test_results, read_reviews, data_tests, read_code_blast_tests, read_exercises, data_implementation_exercise = read_data()

data_linting_result = clean_linting_results(read_linting_results)
data_review_test_results = clean_review_test_results(read_review_test_results)
data_reviews = clean_data_reviewss(read_reviews)
data_exercises = clean_data_exercises(read_exercises)
data_code_blast_tests = clean_data_code_blast_tests(read_code_blast_tests)


In [27]:
# Samenvoegen van de datasets op basis van 'blast_review_id' en 'id'
merged_data = pd.merge(data_review_test_results, data_reviews, left_on='blast_review_id', right_on='id')

# Filteren op de gewenste taal ('test_language')
filtered_data = merged_data[merged_data['test_language'].isin(['nl', 'en'])]

# Beperk de resultaten tot 500 Engelse en 500 Nederlandse
english_data = filtered_data[filtered_data['test_language'] == 'en'].head(500)
dutch_data = filtered_data[filtered_data['test_language'] == 'nl'].head(500)

# Tel het aantal ACCEPTED en DECLINED voor Engelse data
english_counts = english_data['state'].value_counts()

# Tel het aantal ACCEPTED en DECLINED voor Nederlandse data
dutch_counts = dutch_data['state'].value_counts()

# Toon de resultaten
print("Aantal ACCEPTED en DECLINED voor Engelse data:")
print(english_counts)

print("\nAantal ACCEPTED en DECLINED voor Nederlandse data:")
print(dutch_counts)

Aantal ACCEPTED en DECLINED voor Engelse data:
state
DECLINED    313
ACCEPTED    187
Name: count, dtype: int64

Aantal ACCEPTED en DECLINED voor Nederlandse data:
state
DECLINED    273
ACCEPTED    227
Name: count, dtype: int64


In [35]:
merged = pd.merge(data_review_test_results, data_linting_result, left_on='blast_review_id', right_on='blast_review_id')

merged

,blast_review_id,test_id,human_error_message,test_language,file_name,message,type,line,column
0,00001c9e-f8a5-4b92-8c94-b02f2209c968,156effcd-27f9-11ec-abc5-4213e7ee7fac,Je gebruikt echt `DELETE` in je SQL bestand,nl,exercise.sql,Unrecognized statement type.,Error,2,1
1,00001c9e-f8a5-4b92-8c94-b02f2209c968,156effcd-27f9-11ec-abc5-4213e7ee7fac,Je gebruikt echt `DELETE` in je SQL bestand,nl,export.sql,Unexpected token.,Error,167,1
2,00006cbc-1bca-11ec-be5a-4213e7ee7fac,d8cbdd65-bcf5-11ea-a2fb-cec41367f4e7,NaN,nl,/bucket_list.php,"Expected at least 1 space after "".""; 0 found",ERROR,13,77
3,00006cbc-1bca-11ec-be5a-4213e7ee7fac,d8cbdd65-bcf5-11ea-a2fb-cec41367f4e7,NaN,nl,/bucket_list.php,"Expected at least 1 space after "".""; 0 found",ERROR,13,77
4,00006cbc-1bca-11ec-be5a-4213e7ee7fac,d8cbdd65-bcf5-11ea-a2fb-cec41367f4e7,NaN,nl,/bucket_list.php,"Expected at least 1 space after "".""; 0 found",ERROR,13,77
...,...,...,...,...,...,...,...,...,...
2763893,ffff20a0-7dda-43ba-861c-6aed752605c2,321a3324-4302-11ed-863f-4213e7ee7fac,NaN,nl,select01.sql,Unrecognized keyword.,Error,8,16
2763894,ffff20a0-7dda-43ba-861c-6aed752605c2,321a3324-4302-11ed-863f-4213e7ee7fac,NaN,nl,select01.sql,Unexpected token.,Error,8,22
2763895,ffff20a0-7dda-43ba-861c-6aed752605c2,321a3324-4302-11ed-863f-4213e7ee7fac,NaN,nl,select01.sql,Unexpected token.,Error,8,29
2763896,ffff6535-2340-470c-b387-202e8f8edf2b,03d63632-418c-11ed-863f-4213e7ee7fac,NaN,nl,/script.js,Missing semicolon.,error,1,30
